In [20]:
#librerias a importar
#requests es un libreria que nos permite hacer peticiones a apis esta es fundamental para el proyecto
#json es una libreria para modificar el aspecto visual de un json
#datetime es una libreria que contiene valores de tiempo su uso va principalmente relacionado con el acceso a datos
#azurecosmos son librerias para la conecion a la base de datos de cosmos en azure estas nos permiten acceder a la api de la base de datos para hacer consultas
#os es una libreria que nos permite interactuar con el sistema operativo en donde se ejecuta el programa
import json, requests
from datetime import datetime
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import os

In [21]:
#settings es un diccionario de datos que contiene todos los datos para acceder a la base de datos de cosmos en azure
settings = {
    'host': os.environ.get('ACCOUNT_HOST', 'https://comosdavidc.documents.azure.com:443/'),

    #'master_key': os.environ.get('ACCOUNT_KEY', 'HVPRgzkOw8QXjpQtgQ3SNI92DvyJyOgfQfL0K9FXtX1a6KF4ezRFAJ4NiF5xJP6YGTmjW6P0LVDKACDblqdSxA=='),
    'master_key': os.environ.get('ACCOUNT_KEY', 'wZVdL0vlwCNxIhrxzLClhWChJcvpANGTJ3NZz8C7oLBgDjk9BeFnZlI7jFj6T4Hml9dFOMh2xgJqACDbqbAp1A=='),
    'database_id': os.environ.get('COSMOS_DATABASE', 'vulnerabilities0'),
    'container_id': os.environ.get('COSMOS_CONTAINER', 'vulnerabilities'),
}
#aqui se asignan los datos almacenados en settings dentro de variables que van a ser usadas para acceder a la base de datos
HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']
#addInContainer es un metodo utilizado para almacenar la informacion en cosmos crea items en los contenedores y comprueba si existen los recursos
def addInContainer(id):
    
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample

        try:
            container1 = db.create_container(id=id, partition_key=PartitionKey(path='/partitionKey'))
        except exceptions.CosmosResourceExistsError:
            container1 = db.get_container_client(id)

        dataS=getApiShodan(id)
        container1.create_item(body=dataS)
        print("Creation item sucessfull")
    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

In [19]:
#apiCVE es una funcion que consume una api que contiene la informacion relevante del CVE que necesitamos para complementar nuestro programa
#cve es el parametro requerido para hacer la consulta este determinara que sistemas afeta el cve y los datos relevantes del mismo
def apiCVE(cve):
    return requests.get(f"https://cveawg.mitre.org/api/cve/{str(cve)}").json()
#apiShodan es una funcion la cual nos envia datos acerca de la informcaion de los puertos y las vulnerabilidades que shodan tenga almacenados
#ip es el argumento que esta funcion necesita para proceder a realizar la peticion, internamente ya se tiene guardada la key de la api
#los datos retornados son un json el cual contiene informacion relevante acerca de la ip como vulnerabilidades entre otras cosas
def apiShodan(ip):
    key="a1RpAyRaTw7r2WGg6FpxVATNhwnrjynp"
    return requests.get("https://api.shodan.io/shodan/host/"+ip+"?key="+key).json()

In [18]:
#jsonDataVulns es un metodo que usa el metodo anterior de apiCVE para obtener informacion acerca de el cve
#este metodo tambien se encarga de almacenar los datos de forma ordenada en un diccionario de datos
#dentro de sus argumentos pide el cve que se va a evaluar y el score de vulnerabilidad que este cve representa
def jsonDataVulns(cve,score):
    vulnerability={}
    vulnerability[str(cve)]={}
    vulnerability[cve]['systems']=[]
    dataCVE=apiCVE(cve)
    for i in dataCVE["containers"]["cna"]["affected"]:
        data={}
        data['versions']=[]
        for j in i:
            if(j!='versions'):
                data[j]=i[j]
            else:
                for k in i[j]:
                    data[j].append(k['version'])
        vulnerability[cve]['systems'].append(data)
        
    vulnerability[cve]['urls']=[]
    iterator=0
    for urls in dataCVE["containers"]["cna"]["references"]:
        if(iterator==4):
            break
        else:
            iterator+=1
        vulnerability[cve]['urls'].append(urls['url'])
    vulnerability[cve]['vulnerabilityScore']=score
    

    return vulnerability

In [22]:
#getApiShodan es un metodo que nos entrega el item final que queremos crear este metodo llama al metodo jsonDataVulns para obtener la informacion
#ip es el argumento donde recibimos la ip que se requiere utilizar en todo el proceso de evaluacion de esta
#internamente se revisa las vulnerabilidades que tiene se halla el score de esta y se envia para profundizar en el analisis de esta informacion
def getApiShodan(ip):
    datashodan=apiShodan(ip)
    list = []
    vulnsExists=False
    for question in datashodan:
        if(question=='vulns'):
            vulnsExists=True

    if(vulnsExists):
        for cve in datashodan['vulns']:
            truste=False
            for i in datashodan['data']:
                for j in i:
                    if(j=='vulns'):
                        for k in i[j]:
                            if(cve==k):
                                score=i[j][k]['cvss']
                                truste=True
                                break
                    if(truste):
                        break
            list.append(jsonDataVulns(cve,score))
    data={}
    data['id']=ip+" "+datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data['vulnerabilities']=list
    return data

In [ ]:
#list es la lista de ips que contienen la vulnerabilidad bluekeep que el profesor nos entrego
list=[
    "219.75.232.46",
"80.120.238.18",
"14.215.125.26",
"119.3.66.195",
"196.21.83.201",
"218.189.143.54",
"79.159.245.145",
"185.198.75.90",
"80.188.166.75",
"134.175.237.77",
"147.124.212.200",
"118.174.47.121",
"184.151.55.178",
"213.7.220.103",
"152.231.74.206",
"107.85.109.35",
"219.75.232.47",
"172.254.110.44",
"119.194.62.121",
"74.208.229.229"]
#este for es el ciclo que recorre todo el metodo en busca de analizar las diferentes ips enviadas
for i in list:
    print(getApiShodan(i))
    

{'id': '219.75.232.46 2024-03-18 23:20:47', 'vulnerabilities': [{'CVE-2006-6578': {'systems': [{'versions': ['n/a'], 'product': 'n/a', 'vendor': 'n/a'}], 'urls': ['http://securityreason.com/securityalert/2036', 'http://www.securityfocus.com/archive/1/454268/100/0/threaded'], 'vulnerabilityScore': 7.5}}, {'CVE-2009-2521': {'systems': [{'versions': ['n/a'], 'product': 'n/a', 'vendor': 'n/a'}], 'urls': ['https://oval.cisecurity.org/repository/search/definition/oval%3Aorg.mitre.oval%3Adef%3A6508', 'http://support.microsoft.com/default.aspx?scid=kb%3B%5BLN%5D%3BQ975191', 'http://www.us-cert.gov/cas/techalerts/TA09-286A.html', 'https://docs.microsoft.com/en-us/security-updates/securitybulletins/2009/ms09-053'], 'vulnerabilityScore': 5.0}}, {'CVE-2009-4445': {'systems': [{'versions': ['n/a'], 'product': 'n/a', 'vendor': 'n/a'}], 'urls': ['http://securitytracker.com/id?1023387', 'http://soroush.secproject.com/downloadable/iis-semicolon-report.pdf', 'https://exchange.xforce.ibmcloud.com/vulnera

In [ ]:

getApiShodan('134.175.237.77')

In [ ]:

80.188.166.75
134.175.237.77